In [17]:
import pandas as pd
import torch

# 1. Parse RAW CAN log into DataFrame
entries = []
with open('/content/drive/MyDrive/abc.txt', 'r') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) < 10:
            continue
        ts      = float(parts[1])
        node_id = int(parts[3], 16)
        dlc     = int(parts[parts.index('DLC:') + 1])
        entries.append((ts, node_id, dlc))

df = pd.DataFrame(entries, columns=['ts','src','dlc'])
df.sort_values('ts', inplace=True)
df['dt'] = df['ts'].diff().fillna(0.0)

# 2. Feature Engineering: dt + one-hot DLC
one_hot = pd.get_dummies(df['dlc'], prefix='dlc').astype('float32')
features = pd.concat([df[['dt']], one_hot], axis=1).values.astype('float32')

# 3. Build the four lists (plus placeholder labels if needed)
src     = df['src'].tolist()                                    # source node IDs
dst     = src                                                   # destination (self‑edges)
ts_list = df['ts'].tolist()                                     # timestamps
x_feats = [torch.tensor(f, dtype=torch.float32) for f in features]  # feature vectors
y       = [0] * len(df)                                         # placeholder labels

# 4. Print the first few entries of each list
print("First 5 source nodes (src):", src[:5])
print("First 5 destination nodes (dst):", dst[:5])
print("First 5 timestamps (ts_list):", ts_list[:5])
print("First 5 feature vectors (x_feats):")
for vec in x_feats[:5]:
    print(" ", vec)


First 5 source nodes (src): [544, 1201, 688, 357, 544]
First 5 destination nodes (dst): [544, 1201, 688, 357, 544]
First 5 timestamps (ts_list): [0.0, 0.000868, 0.001212, 0.001452, 0.001782]
First 5 feature vectors (x_feats):
  tensor([0., 0., 1.])
  tensor([8.6800e-04, 0.0000e+00, 1.0000e+00])
  tensor([3.4400e-04, 1.0000e+00, 0.0000e+00])
  tensor([2.4000e-04, 0.0000e+00, 1.0000e+00])
  tensor([3.3000e-04, 0.0000e+00, 1.0000e+00])
